In [1]:
import os
import numpy as np
import pandas as pd
import torch
from scipy.sparse import csr_matrix
from sklearn import preprocessing as pp, cluster as cl, metrics as mt, decomposition as dc, neighbors as nb, pipeline
import humanfriendly as hf
import cProfile

os.chdir("C:\\Users\\ndgig\\Repositories\\phrase_foundry")
print(os.getcwd())

%load_ext autoreload
%autoreload 2

from phrase_foundry import PhraseFoundry, ApproxPhraseFoundry

C:\Users\ndgig\Repositories\phrase_foundry


In [ ]:
splits = {
    "train": "train_consumer_complaints.csv",
    "test": "test_consumer_complaints.csv",
}
df = pd.read_csv(
    "hf://datasets/Johnade/consumer_complaints_cfpb/" + splits["train"]
).sample(75000, random_state=32)

docs = df["consumer_complaint_narrative"].to_list()

docs[:5]

['The credit bureaus did not fix any dispute or investigation. The automative system continues to fail and not do anything about the problem.',
 "I first noticed that my online university of XXXX account was audited when logging in after 4 years, due to data breaches on me and my kids information. When printing out the loans that I supposedly taken out I went to the bank I was banking with at the time because I knew they never deposited any student loans into my account. When retrieving the old bank STATEMENTS from XXXX XXXX XXXX I became stressed to the max due to the account showing XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX FL then XXXX XXXX XXXX XXXX fL and over 20 check cards being used and many deposits being made and the acct was reopened without my consent after closing it 6 months prior to it being reopened without my consent. Then I noticed the acct showing financial cons data etc removing money from an altered acct I knew nothing about. The withdrawls and deposits and

In [ ]:
if "model" in locals():
    del model
model = ApproxPhraseFoundry(
    # "sentence-transformers/all-mpnet-base-v2",
    "sentence-transformers/paraphrase-MiniLM-L3-v2",
    n_pca_components=64,
    n_pca_training_samples=int(8e6),
    device="cuda",
    amp=True,
    amp_dtype=torch.bfloat16,
    quantize_embeds=True,
)
model

In [4]:
samp_idx, ngrams, ngram_vecs = model.encode_extract(
    docs,
    batch_size=512,
    ngram_range=(4, 6),
)

ngram_vecs

Encoding:   0%|          | 0/17 [00:00<?, ?it/s]

c:\Users\ndgig\miniforge3\envs\nlp1\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


RuntimeError: PCA training failed to complete. Reduce `n_pca_training_samples`.

In [10]:
ngram_vecs.shape

(6052104, 64)

In [ ]:
model.pca_.n_samples_seen_

2214716

In [20]:
docs[1], ngrams[samp_idx == 1].tolist()

("I first noticed that my online university of XXXX account was audited when logging in after 4 years, due to data breaches on me and my kids information. When printing out the loans that I supposedly taken out I went to the bank I was banking with at the time because I knew they never deposited any student loans into my account. When retrieving the old bank STATEMENTS from XXXX XXXX XXXX I became stressed to the max due to the account showing XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX FL then XXXX XXXX XXXX XXXX fL and over 20 check cards being used and many deposits being made and the acct was reopened without my consent after closing it 6 months prior to it being reopened without my consent. Then I noticed the acct showing financial cons data etc removing money from an altered acct I knew nothing about. The withdrawls and deposits and counter credits being made in different states and cities when i could never be in 2 states at the same time. When checking the emails attached

In [12]:
hf.format_size(ngram_vecs.nbytes)

'774.67 MB'

In [1]:
def search(
    queries: list[str],
    query_vecs: np.ndarray,
    ngrams: np.ndarray = ngrams,
    ngram_vecs: np.ndarray = ngram_vecs,
    samp_idx: np.ndarray = samp_idx,
    radius: float = 0.5,
    metric: str = "cosine",
):
    search_index = nb.NearestNeighbors(radius=radius, metric=metric).fit(ngram_vecs)
    dists, idx = search_index.radius_neighbors(query_vecs, return_distance=True)
    rankings = [np.argsort(d) for d in dists]
    idx = [i[r] for i, r in zip(idx, rankings)]
    return {
        queries[i]: (ngrams[idx[i]], np.unique(samp_idx[idx[i]]))
        for i in range(len(queries))
    }


queries = [
    "close my account",
    "credit card fraud",
    "debt collection",
    "identity theft",
    "mortgage fraud",
    "overdraft fees",
    "payday loan",
    "student loan",
    "unauthorized transaction",
    "vehicle loan",
]
query_vecs = model.encode_queries(queries)
results = search(queries, query_vecs, ngram_vecs=ngram_vecs, radius=0.2)
results

NameError: name 'ngrams' is not defined

In [17]:
results["unauthorized transaction"][0].tolist()

['a unauthorized transaction for',
 'saw a unauthorized transaction for',
 'unauthorized transactions using my',
 'with unauthorized transactions the',
 'issue with unauthorized transactions',
 'the unauthorized transactions (',
 'unauthorized transactions on my',
 'experienced several unauthorized transactions']